In [335]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from ast import literal_eval
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import os
import sympy as sym
from sympy.parsing import mathematica
import numpy as np

In [336]:

RESULTS_DIR = "paper_results"
benchmarks = [
    "rational_2",
    "sqrt",    
    "rational",
    "rational_sqrt",   
]

benchmarks_id= {
    
    "rational":'a',
    "rational_2":'b',
    "sqrt":'c',
    "rational_sqrt":'d',
}


In [337]:
def process_output(row):
    try:
        if (type(row) == list) and (len(row) > 0):
            row = np.asarray(row)
            result = (pd.to_numeric(row.mean() - row.std()), pd.to_numeric(row.mean() + row.std()))
            print("Returning from: Check if row is a non-empty list")
            return result
    except:
        pass

    try:
        if (type(row) == list) and (len(row) == 0):
            result = (None, None)
            print("Returning from: Check if row is an empty list")
            return result
    except:
        pass

    try:
        row = float(row)
        result = (row, row)
        print("Returning from: Convert row to float")
        return result
    except:
        pass

    try:
        result = pd.to_numeric(literal_eval(str(row).strip()))
        print("Returning from: Use literal_eval on row")
        return result
    except Exception as e:
        pass

    try:
        for w in ['integrate', 'boole', 'throw', 'complex']:
            if w in row.lower():
                result = (None, None)
                print("Returning from: Parse and evaluate row as a symbolic expression (contains forbidden words)")
                return result
        
        row = row.replace("[", '(').replace("]", ")").replace("L", "l").replace("E(p_) =", "").replace("^", "**").strip()
        row = sym.parse_expr(row).evalf()
        result = (row, row)
        print("Returning from: Parse and evaluate row as a symbolic expression")
        return result
    except Exception as e:
        print(e, row)
    
    print("Returning from: Default case (None, None)")
    return (None, None)

In [338]:
def process_time(row):
    if pd.isnull(row):
        return None

    try:
        row = float(row)
        return row
    except:
        pass

    try:
        return literal_eval(str(row).strip())[0]
    except Exception as e:
        pass
        
    except Exception as e:
        print(e, row)
    
        
    return None

In [339]:

def process_benchmark(benchmark):
    df_list = []
    for f_name in os.listdir(os.path.join(RESULTS_DIR, benchmark)):
        tool_name =  f_name.split("_")[-2]
        # if tool_name != "mathematica":
        #     continue
        df_ = pd.read_csv(os.path.join(RESULTS_DIR, benchmark, f_name))
        df_['tool'] = tool_name
        df_['details'] = df_['details'].apply(literal_eval)
        if tool_name == 'volesti': 
            df_['output'] = df_['details'].apply(lambda d: list([x['output'] for x in d]))
        df_list.append(df_[['index', 'tool','output', 'time', 'details', 'formula']])
    df = pd.concat(df_list, ignore_index=True)

    df['raw_output'] = df['output']
    df['output'] = df['output'].apply(process_output)
    df['time'] = df['time'].apply(process_time)
    
    df['lower'], df['upper'] = df['output'].apply(lambda o: o[0]),df['output'].apply(lambda o: o[1])
    
    df['error'] = df['upper']-df['lower']
    df['error_norm'] = MinMaxScaler().fit_transform(df['error'].values.reshape(-1,1))
    df['benchmark'] = benchmark
    df['is_solved'] = (~pd.isnull(df['error']))
    df['is_timeout'] = df['time'].apply(lambda t: t > 3550)
    return df

In [340]:
df_b_list = []
for b in benchmarks:
    df_b_list.append(process_benchmark(b))

df_b = pd.concat(df_b_list)

Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
Returning from: Convert row to float
R

In [341]:
def calcPlotID(bench):
    if bench.benchmark == 'rational':
        return 'A'+str(bench['index']+1)
    elif bench.benchmark == 'rational_2':
        return 'A'+str(bench['index']+30+1)
    if bench.benchmark == 'sqrt':
        return 'B'+str(bench['index']+1)
    elif bench.benchmark == 'rational_sqrt':
        return 'B'+str(bench['index']+30+1)
    
    return 0


df_b['plotID'] = df_b.apply(calcPlotID, axis=1)
df_b['plotIndex'] = df_b['plotID'].apply(lambda id: int(id[1:]) if id[0]=='A' else int(id[1:])+60)
df_b.head()

,index,tool,output,time,details,formula,raw_output,lower,upper,error,error_norm,benchmark,is_solved,is_timeout,plotID,plotIndex
0,0,latte,"(nan, nan)",0.078246,[],4.31*((1.03*x*x + 1.1*y*y + 0.48*x*y + 8.88*x ...,NaN,NaN,NaN,NaN,NaN,rational_2,False,False,A31,31
1,1,latte,"(nan, nan)",0.058846,[],5.28*((6.82*x*x + 5.6*y*y + 1.32*x*y + 8.69*x ...,NaN,NaN,NaN,NaN,NaN,rational_2,False,False,A32,32
2,2,latte,"(nan, nan)",0.053086,[],4.14*((5.72*x*x + 3.43*y*y + 7.83*x*y + 4.6*x ...,NaN,NaN,NaN,NaN,NaN,rational_2,False,False,A33,33
3,3,latte,"(nan, nan)",0.048246,[],9.59*((5.79*x*x + 1.86*y*y + 2.65*x*y + 4.62*x...,NaN,NaN,NaN,NaN,NaN,rational_2,False,False,A34,34
4,4,latte,"(nan, nan)",0.052024,[],5.88*((7.82*x*x + 5.83*y*y + 7.55*x*y + 3.35*x...,NaN,NaN,NaN,NaN,NaN,rational_2,False,False,A35,35


In [342]:
# Extracted IDs from the table
sampled_ids = ['A12', 'A17', 'A22', 'A30', 'A41', 'A49', 'A53', 'B11', 'B17', 'B18', 'B24', 'B32', 'B35', 'B38']


In [343]:
# \newcommand{\ns}{{\color{red}N/S}}%
# \newcommand{\timeout}{{\color{red}\textbf{TO}}}% Timeout symbol
# \newcommand{\complexoutput}{{\color{red}\textbf{CX}}}% Complex output symbol
# \newcommand{\symbolicoutput}{{\color{red}\textbf{SY}}}% Symbolic output symbol
# \newcommand{\toolerror}{{\color{red}\textbf{ER}}}% Tool error symbol

In [344]:
df = df_b[df_b.benchmark.isin(['rational_sqrt', 'sqrt'])]
# df = df_b[df_b.plotID.isin(sampled_ids)]
# df = df_b[df_b.benchmark.isin(['rational', 'rational_2'])]

df_l = {}

for i, row in df.iterrows():
    if row['plotID'] not in df_l:
        df_l[row['plotID']] = {
            'plotIndex': row['plotIndex']
        }
        
    if row.tool in ['gubpi']:
        if pd.isnull(row.upper) or pd.isna(row.upper) or "NaN" in str(row.upper):
            df_l[row['plotID']][f"{row.tool}lower"] = r'\toolerror'
            df_l[row['plotID']][f"{row.tool}upper"] = r'\toolerror'
        else:
            df_l[row['plotID']][f"{row.tool}lower"] = round(row.lower, 6)
            df_l[row['plotID']][f"{row.tool}upper"] = round(row.upper, 6)
    
    if row.tool in ['wmilp']:
        df_l[row['plotID']][f"{row.tool}lower"] = round(row.lower, 6)
        df_l[row['plotID']][f"{row.tool}upper"] = round(row.upper, 6)
        
    
    if row.benchmark in ['rational_sqrt', 'sqrt']:
        if row.tool in ['volesti']:
            df_l[row['plotID']][f"{row.tool}lower"] = r'\ns'
            df_l[row['plotID']][f"{row.tool}upper"] = r'\ns'
        
        if row.tool in ["latte"]:
            df_l[row['plotID']][f"{row.tool}"] = r"\ns"
    elif row.benchmark in ['rational', 'rational_2']:
        if row.tool in ['volesti']:
            df_l[row['plotID']][f"{row.tool}lower"] = round(row.lower, 6)
            df_l[row['plotID']][f"{row.tool}upper"] = round(row.upper, 6)
        
        if row.tool in ["latte"]:
            df_l[row['plotID']][f"{row.tool}"] = r'\toolerror'
        

    if row.tool in ["psi"]:
        if pd.isnull(row.upper):
            if row.is_timeout:
                df_l[row['plotID']][f"{row.tool}"] = r'\timeout'
            elif 'integrate' in str(row.raw_output).lower():
                df_l[row['plotID']][f"{row.tool}"] = r'\symbolicoutput'
            else:
                df_l[row['plotID']][f"{row.tool}"] = r'\toolerror'
        else:    
            df_l[row['plotID']][f"{row.tool}"] = round(row.upper, 6)

    if row.tool in ["mathematica"]:
        if pd.isnull(row.upper):
            if 'complex' in str(row.raw_output).lower():
                df_l[row['plotID']][f"{row.tool}"] = r'\complexoutput'
            elif 'integrate' in str(row.raw_output).lower():
                df_l[row['plotID']][f"{row.tool}"] = r'\symbolicoutput'
            elif row.is_timeout:
                df_l[row['plotID']][f"{row.tool}"] = r'\timeout'
            else:
                df_l[row['plotID']][f"{row.tool}"] = r'\toolerror'
        else:    
            df_l[row['plotID']][f"{row.tool}"] = round(row.upper, 6)


    

df_l = (pd.DataFrame(df_l).T).sort_values('plotIndex')[[
    'wmilplower', 
    'wmilpupper', 
    'gubpilower', 'gubpiupper', 
    'volestilower', 'volestiupper',
    'latte', 
    'psi',
    'mathematica'
    ]]

# df_l = df_l.fillna(r"\xmark")

df_l

,wmilplower,wmilpupper,gubpilower,gubpiupper,volestilower,volestiupper,latte,psi,mathematica
B1,8.189794,8.289763,\toolerror,\toolerror,\ns,\ns,\ns,8.273899,8.273899
B2,7.251466,7.351412,0.0,7.300108,\ns,\ns,\ns,7.300024,7.300024
B3,7.988964,8.088911,0.0,8.041192,\ns,\ns,\ns,\symbolicoutput,8.041033
B4,5.316964,5.41695,1.000513,5.368105,\ns,\ns,\ns,\symbolicoutput,\complexoutput
B5,6.852915,6.95279,0.0,6.905365,\ns,\ns,\ns,6.905314,6.905314
B6,2.506375,2.606369,2.552386,2.556905,\ns,\ns,\ns,\symbolicoutput,\complexoutput
B7,8.005613,8.105583,0.0,8.056841,\ns,\ns,\ns,8.056740,8.05674
B8,4.362097,4.462019,1.481235,4.414413,\ns,\ns,\ns,\symbolicoutput,4.411721
B9,1.427785,1.527674,1.475519,1.47654,\ns,\ns,\ns,\symbolicoutput,1.47603
B10,8.806871,8.906846,0.0,8.855746,\ns,\ns,\ns,\symbolicoutput,8.855606


In [345]:
df_l.to_latex(os.path.join(RESULTS_DIR, 'tables', 'benchB.tex'))